In [1]:
import numpy as np
import os
import librosa
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import pickle

In [16]:
import wave
import numpy as np
def corte(pasta):
    # definir o tempo de corte em segundos
    start_time = 1
    end_time = 5

    # definir as pastas de entrada e saída
    input_folder = pasta+'/'
    output_folder = pasta+'Cortado'+'/'

    # criar a pasta de saída, se não existir
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # percorrer todos os arquivos de áudio na pasta de entrada
    for filename in os.listdir(input_folder):
        if filename.endswith('.wav') or filename.endswith('.ogg'):
            file_path = os.path.join(input_folder, filename)

            # carregar o arquivo de áudio
            y, sr = librosa.load(file_path, sr=None)

            # calcular os índices de início e fim do trecho a ser cortado
            start_idx = int(start_time * sr)
            end_idx = int(end_time * sr)

            # cortar o trecho do áudio
            y_cut = y[start_idx:end_idx]

            # salvar o trecho cortado em um novo arquivo
            output_path = os.path.join(output_folder, filename)
            #librosa.output.write_wav(output_path, y_cut, sr)
            with wave.open(output_path, 'wb') as wav_file:
                # Definir os parâmetros do arquivo WAV
                wav_file.setnchannels(1)  # Mono
                wav_file.setsampwidth(2)  # 16 bits
                wav_file.setframerate(sr)  # 44.1 kHz
                # Escrever os dados de áudio no arquivo WAV
                wav_file.writeframes(y_cut.tobytes())


In [1]:
classes = ['ChoroTudo', 'laugh', 'Noise','silence']
for i in classes:
    corte(i)

NameError: name 'corte' is not defined

In [39]:
# definir as classes dos sons a serem aprendidos
classes = ['Choro', 'Risada', 'Barulho','Silencio']
#classes = ['belly_pain','burping','discomfort','hungry','tired']
# carregar os arquivos de áudio e extrair recursos
features = []
labels = []
for c in classes:
    path = 'Dataset_Microfone/' + c + '/'
    for filename in os.listdir(path):
        if filename.endswith('.wav') or filename.endswith('.ogg'):
            file_path = os.path.join(path, filename)
            x, sr = librosa.load(file_path)
            mfccs = librosa.feature.mfcc(y=x, sr=sr, n_mfcc=90)
            # Aplica a normalização das features
            scaler = StandardScaler()
            mfccs_norm = scaler.fit_transform(mfccs.T)

            # Transpõe novamente as features normalizadas para o formato original
            mfccs_norm = mfccs_norm.T

            features.append(np.mean(mfccs, axis=1))
            labels.append(c)

# dividir o conjunto de dados em treinamento e teste 0.2
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42)

# criar um classificador MLP
#100
clf = MLPClassifier(hidden_layer_sizes=(300,), max_iter=700, early_stopping=True,validation_fraction=0.3, verbose=True)

# treinar o classificador
clf.fit(X_train, y_train)

# fazer previsões no conjunto de teste
y_pred = clf.predict(X_test)

# avaliar a precisão do classificador
accuracy = accuracy_score(y_test, y_pred)
print('Precisão do classificador: {:.2f}%'.format(accuracy * 100))

Iteration 1, loss = 10.83907343
Validation score: 0.406593
Iteration 2, loss = 6.49301936
Validation score: 0.384615
Iteration 3, loss = 3.58363334
Validation score: 0.263736
Iteration 4, loss = 4.65804208
Validation score: 0.494505
Iteration 5, loss = 2.42755124
Validation score: 0.439560
Iteration 6, loss = 1.79033709
Validation score: 0.428571
Iteration 7, loss = 1.69909765
Validation score: 0.483516
Iteration 8, loss = 1.49038622
Validation score: 0.703297
Iteration 9, loss = 0.79298530
Validation score: 0.615385
Iteration 10, loss = 1.16530234
Validation score: 0.725275
Iteration 11, loss = 0.70010671
Validation score: 0.835165
Iteration 12, loss = 0.38218578
Validation score: 0.703297
Iteration 13, loss = 0.70826006
Validation score: 0.670330
Iteration 14, loss = 0.77010104
Validation score: 0.857143
Iteration 15, loss = 0.30191043
Validation score: 0.824176
Iteration 16, loss = 0.30364745
Validation score: 0.824176
Iteration 17, loss = 0.32768536
Validation score: 0.835165
Itera

In [40]:
# salvar o modelo treinado
with open('Modelos/modelo_Tipos_De_ChoroVMicrofone.pkl', 'wb') as f:
    pickle.dump(clf, f)


In [41]:
# carregar o modelo treinado
with open('Modelos/modelo_Tipos_De_ChoroVMicrofone.pkl', 'rb') as f:
    clf = pickle.load(f)

In [43]:
# fazer previsões em novos dados
x, sr = librosa.load('Dataset_Microfone/Microfone_Teste_Aleatorio.wav')
mfccs = librosa.feature.mfcc(y=x, sr=sr, n_mfcc=90)
features = np.mean(mfccs, axis=1)
y_pred = clf.predict([features])
print('O som é da classe:', y_pred[0])

O som é da classe: Choro
